In [1]:
# TODO

# 1.

In [7]:
import combobgen
from combobgen import *

# print("Путь к модулю:", combobgen.__file__)
# print("Загрузчик:", combobgen.__loader__)

# print(dir(combobgen))

# ТЕСТЫ ДЛЯ АЛГОРИТМОВ ГЕНЕРАЦИИ ПЕРЕСТАНОВОК

In [3]:
def test_permutation_uniformity(algorithm: Callable, n: int, trials: int = 10000) -> List[tuple]:
    """
    Тестирует равномерность распределения перестановок.
    
    Args:
        algorithm: функция алгоритма перестановки
        n: размер перестановки
        trials: количество испытаний
    
    Returns:
        список кортежей (metric_name, value)
    """
    if n > 5:  # Ограничиваем для больших n из-за n! перестановок
        return _test_position_uniformity_permutation(algorithm, n, trials)
    else:
        return _test_full_permutation_uniformity(algorithm, n, trials)


def _test_full_permutation_uniformity(algorithm: Callable, n: int, trials: int) -> List[tuple]:
    """Тестирует равномерность всех возможных перестановок (для малых n)."""
    permutation_counts = defaultdict(int)
    
    for _ in range(trials):
        if algorithm.__name__ == 'fisher_yates_shuffle':
            arr = list(range(n))
            algorithm(arr)
            perm = tuple(arr)
        else:
            perm = tuple(algorithm(n))
        permutation_counts[perm] += 1
    
    # Вычисляем статистики
    expected_count = trials / math.factorial(n)
    chi_squared = sum((count - expected_count) ** 2 / expected_count 
                     for count in permutation_counts.values())
    
    return [
        ('total_permutations_found', len(permutation_counts)),
        ('expected_permutations', math.factorial(n)),
        ('chi_squared', round(chi_squared, 4)),
        ('expected_count_per_permutation', round(expected_count, 2)),
        ('coverage_ratio', round(len(permutation_counts) / math.factorial(n), 4)),
        ('uniformity_quality', 'excellent' if chi_squared < math.factorial(n) * 0.5 
                              else 'good' if chi_squared < math.factorial(n) 
                              else 'poor')
    ]


def _test_position_uniformity_permutation(algorithm: Callable, n: int, trials: int) -> List[tuple]:
    """Тестирует равномерность распределения элементов по позициям."""
    position_counts = defaultdict(lambda: defaultdict(int))
    
    for _ in range(trials):
        if algorithm.__name__ == 'fisher_yates_shuffle':
            arr = list(range(n))
            algorithm(arr)
            perm = arr
        else:
            perm = algorithm(n)
        
        for pos, val in enumerate(perm):
            position_counts[val][pos] += 1
    
    # Вычисляем статистики
    expected_count = trials / n
    chi_squared_by_element = {}
    
    for element in range(n):
        chi_sq = sum((position_counts[element][pos] - expected_count) ** 2 / expected_count 
                    for pos in range(n))
        chi_squared_by_element[element] = chi_sq
    
    avg_chi_squared = sum(chi_squared_by_element.values()) / n
    
    return [
        ('expected_count_per_position', round(expected_count, 2)),
        ('avg_chi_squared', round(avg_chi_squared, 4)),
        ('max_chi_squared', round(max(chi_squared_by_element.values()), 4)),
        ('min_chi_squared', round(min(chi_squared_by_element.values()), 4)),
        ('position_uniformity', 'excellent' if avg_chi_squared < n * 0.5 
                               else 'good' if avg_chi_squared < n 
                               else 'poor')
    ]


def test_permutation_properties(algorithm: Callable, n: int, trials: int = 1000) -> List[tuple]:
    """
    Тестирует свойства перестановок: уникальность элементов, полнота, etc.
    
    Args:
        algorithm: функция алгоритма перестановки
        n: размер перестановки
        trials: количество испытаний
    
    Returns:
        список кортежей (metric_name, value)
    """
    valid_permutations = 0
    duplicate_elements = 0
    missing_elements = 0
    
    for _ in range(trials):
        if algorithm.__name__ == 'fisher_yates_shuffle':
            arr = list(range(n))
            algorithm(arr)
            perm = arr
        else:
            perm = algorithm(n)
        
        # Проверяем уникальность элементов
        if len(set(perm)) != len(perm):
            duplicate_elements += 1
        
        # Проверяем полноту (все элементы от 0 до n-1)
        if set(perm) != set(range(n)):
            missing_elements += 1
        
        # Проверяем, что это валидная перестановка
        if len(perm) == n and set(perm) == set(range(n)):
            valid_permutations += 1
    
    return [
        ('valid_permutations', valid_permutations),
        ('duplicate_elements_count', duplicate_elements),
        ('missing_elements_count', missing_elements),
        ('correctness_ratio', round(valid_permutations / trials, 6)),
        ('algorithm_correctness', 'perfect' if valid_permutations == trials else 'broken')
    ]


def test_permutation_performance(algorithm: Callable, n: int, trials: int = 100) -> List[tuple]:
    """
    Тестирует производительность алгоритмов перестановок.
    
    Args:
        algorithm: функция алгоритма
        n: размер перестановки
        trials: количество запусков
    
    Returns:
        список кортежей (metric_name, value)
    """
    times = []
    
    for _ in range(trials):
        if algorithm.__name__ == 'fisher_yates_shuffle':
            test_data = list(range(n))
            start_time = time.time()
            algorithm(test_data)
            end_time = time.time()
        else:
            start_time = time.time()
            algorithm(n)
            end_time = time.time()
        
        times.append(end_time - start_time)
    
    avg_time = sum(times) / len(times)
    std_time = (sum((t - avg_time)**2 for t in times) / len(times)) ** 0.5
    
    return [
        ('avg_time_ms', round(avg_time * 1000, 6)),
        ('min_time_ms', round(min(times) * 1000, 6)),
        ('max_time_ms', round(max(times) * 1000, 6)),
        ('std_time_ms', round(std_time * 1000, 6)),
        ('throughput_ops_per_sec', round(1 / avg_time, 2)),
        ('time_complexity_estimate', f'O(n)' if avg_time < n * 1e-6 else f'O(n log n)' if avg_time < n * math.log(n) * 1e-6 else 'O(n²)+')
    ]


# ТЕСТЫ ДЛЯ АЛГОРИТМОВ ВЫБОРКИ K ИЗ N

In [4]:
def test_sampling_uniformity(algorithm: Callable, n: int, k: int, trials: int = 10000) -> List[tuple]:
    """
    Тестирует равномерность выборки k элементов из n.
    
    Args:
        algorithm: функция алгоритма выборки
        n: размер популяции
        k: размер выборки
        trials: количество испытаний
    
    Returns:
        список кортежей (metric_name, value)
    """
    population = list(range(n))
    element_counts = Counter()
    
    for _ in range(trials):
        sample = algorithm(population, k)
        for item in sample:
            element_counts[item] += 1
    
    # Ожидаемое количество появлений каждого элемента
    expected_count = trials * k / n
    
    # Вычисляем хи-квадрат статистику
    chi_squared = sum((count - expected_count) ** 2 / expected_count 
                     for count in element_counts.values())
    
    # Проверяем, что все элементы были выбраны
    elements_never_selected = n - len(element_counts)
    
    return [
        ('expected_count_per_element', round(expected_count, 2)),
        ('chi_squared', round(chi_squared, 4)),
        ('elements_never_selected', elements_never_selected),
        ('total_samples_taken', trials * k),
        ('unique_elements_sampled', len(element_counts)),
        ('sampling_uniformity', 'excellent' if chi_squared < n * 0.5 
                               else 'good' if chi_squared < n 
                               else 'poor')
    ]


def test_sampling_properties(algorithm: Callable, n: int, k: int, trials: int = 1000) -> List[tuple]:
    """
    Тестирует свойства выборки: размер, уникальность, корректность.
    
    Args:
        algorithm: функция алгоритма выборки
        n: размер популяции
        k: размер выборки
        trials: количество испытаний
    
    Returns:
        список кортежей (metric_name, value)
    """
    population = list(range(n))
    correct_size_count = 0
    duplicate_elements_count = 0
    invalid_elements_count = 0
    
    for _ in range(trials):
        sample = algorithm(population, k)
        
        # Проверяем размер выборки
        if len(sample) == k:
            correct_size_count += 1
        
        # Проверяем уникальность элементов
        if len(set(sample)) != len(sample):
            duplicate_elements_count += 1
        
        # Проверяем, что все элементы из исходной популяции
        if not all(item in population for item in sample):
            invalid_elements_count += 1
    
    return [
        ('correct_size_ratio', round(correct_size_count / trials, 6)),
        ('duplicate_elements_count', duplicate_elements_count),
        ('invalid_elements_count', invalid_elements_count),
        ('expected_sample_size', k),
        ('algorithm_correctness', 'perfect' if correct_size_count == trials and 
                                             duplicate_elements_count == 0 and 
                                             invalid_elements_count == 0 else 'broken')
    ]


def test_sampling_coverage(algorithm: Callable, n: int, k: int, trials: int = 1000) -> List[tuple]:
    """
    Тестирует покрытие элементов при выборке.
    
    Args:
        algorithm: функция алгоритма выборки
        n: размер популяции
        k: размер выборки
        trials: количество испытаний
    
    Returns:
        список кортежей (metric_name, value)
    """
    population = list(range(n))
    all_selected = set()
    selection_counts = Counter()
    
    for _ in range(trials):
        sample = algorithm(population, k)
        for item in sample:
            all_selected.add(item)
            selection_counts[item] += 1
    
    # Теоретическая вероятность НЕ быть выбранным за trials испытаний
    prob_not_selected = ((n - k) / n) ** trials
    expected_never_selected = n * prob_not_selected
    
    return [
        ('total_unique_elements_selected', len(all_selected)),
        ('coverage_ratio', round(len(all_selected) / n, 6)),
        ('elements_never_selected', n - len(all_selected)),
        ('expected_never_selected', round(expected_never_selected, 2)),
        ('min_selection_count', min(selection_counts.values()) if selection_counts else 0),
        ('max_selection_count', max(selection_counts.values()) if selection_counts else 0),
        ('coverage_quality', 'excellent' if len(all_selected) / n > 0.95 
                            else 'good' if len(all_selected) / n > 0.80 
                            else 'poor')
    ]


def test_sampling_performance(algorithm: Callable, n: int, k: int, trials: int = 100) -> List[tuple]:
    """
    Тестирует производительность алгоритмов выборки.
    
    Args:
        algorithm: функция алгоритма
        n: размер популяции
        k: размер выборки
        trials: количество запусков
    
    Returns:
        список кортежей (metric_name, value)
    """
    population = list(range(n))
    times = []
    
    for _ in range(trials):
        start_time = time.time()
        algorithm(population, k)
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = sum(times) / len(times)
    std_time = (sum((t - avg_time)**2 for t in times) / len(times)) ** 0.5
    
    return [
        ('avg_time_ms', round(avg_time * 1000, 6)),
        ('min_time_ms', round(min(times) * 1000, 6)),
        ('max_time_ms', round(max(times) * 1000, 6)),
        ('std_time_ms', round(std_time * 1000, 6)),
        ('throughput_ops_per_sec', round(1 / avg_time, 2)),
        ('time_complexity_estimate', f'O(n)' if avg_time < n * 1e-6 else f'O(n log n)' if avg_time < n * math.log(n) * 1e-6 else 'O(n²)+')
    ]

# Main

In [5]:
random.seed(int(4))
    
print("=== ТЕСТЫ АЛГОРИТМОВ ПЕРЕСТАНОВОК ===")

# Настройка для тестов перестановок
permutation_tests = {
    "uniformity": test_permutation_uniformity,
    "properties": test_permutation_properties,
    "performance": test_permutation_performance,
}

permutation_algs = {
    "Fisher-Yates": fisher_yates_shuffle,
    "Generate-Permutation": generate_random_permutation,
}

permutation_test_cases = {
    "uniformity": {
        "small_case": {"n": 4},
        "medium_case": {"n": 10},
    },
    "properties": {
        "small_case": {"n": 5},
        "large_case": {"n": 100},
    },
    "performance": {
        "small_case": {"n": 100},
        "medium_case": {"n": 1000},
        "large_case": {"n": 10000},
    },
}

for test_name in permutation_tests.keys():
    print(f"\n{'='*60}")
    print(f"ТЕСТ ПЕРЕСТАНОВОК: {test_name.upper()}")
    print(f"{'='*60}")
    
    for alg_name in permutation_algs.keys():
        print(f"\n{'-'*40}")
        print(f"АЛГОРИТМ: {alg_name}")
        print(f"{'-'*40}")
        
        for case_name, params in permutation_test_cases[test_name].items():
            print(f"\n{case_name}: {list(params.values())}")
            
            try:
                results = permutation_tests[test_name](permutation_algs[alg_name], **params)
                for metric_name, value in results:
                    print(f"  {metric_name}: {value}")
            except Exception as e:
                print(f"  ОШИБКА: {e}")

print("\n\n=== ТЕСТЫ АЛГОРИТМОВ ВЫБОРКИ ===")

# Настройка для тестов выборки
sampling_tests = {
    "uniformity": test_sampling_uniformity,
    "properties": test_sampling_properties,
    "coverage": test_sampling_coverage,
    "performance": test_sampling_performance,
}

sampling_algs = {
    "Reservoir-Sampling": reservoir_sampling_list,
}

sampling_test_cases = {
    "uniformity": {
        "small_case": {"n": 20, "k": 5},
        "medium_case": {"n": 100, "k": 10},
    },
    "properties": {
        "basic_case": {"n": 50, "k": 10},
        "edge_case": {"n": 10, "k": 9},
    },
    "coverage": {
        "low_coverage": {"n": 100, "k": 5},
        "high_coverage": {"n": 20, "k": 15},
    },
    "performance": {
        "small_case": {"n": 1000, "k": 100},
        "medium_case": {"n": 10000, "k": 1000},
        "large_case": {"n": 100000, "k": 10000},
    },
}

for test_name in sampling_tests.keys():
    print(f"\n{'='*60}")
    print(f"ТЕСТ ВЫБОРКИ: {test_name.upper()}")
    print(f"{'='*60}")
    
    for alg_name in sampling_algs.keys():
        print(f"\n{'-'*40}")
        print(f"АЛГОРИТМ: {alg_name}")
        print(f"{'-'*40}")
        
        for case_name, params in sampling_test_cases[test_name].items():
            print(f"\n{case_name}: n={params['n']}, k={params['k']}")
            
            try:
                results = sampling_tests[test_name](sampling_algs[alg_name], **params)
                for metric_name, value in results:
                    print(f"  {metric_name}: {value}")
            except Exception as e:
                print(f"  ОШИБКА: {e}")

print("\n=== ПРОВЕРКА КОРРЕКТНОСТИ ===")

# Быстрая проверка корректности
for _ in range(10):
    # Тест Fisher-Yates
    arr = list(range(10))
    fisher_yates_shuffle(arr)
    assert len(set(arr)) == 10, "Fisher-Yates: дублирующиеся элементы"
    assert set(arr) == set(range(10)), "Fisher-Yates: неполная перестановка"
    
    # Тест Reservoir Sampling
    sample = reservoir_sampling_list(list(range(100)), 15)
    assert len(sample) == 15, f"Reservoir: неверный размер выборки {len(sample)}"
    assert len(set(sample)) == 15, "Reservoir: дублирующиеся элементы"

print("Все тесты пройдены успешно!")

=== ТЕСТЫ АЛГОРИТМОВ ПЕРЕСТАНОВОК ===

ТЕСТ ПЕРЕСТАНОВОК: UNIFORMITY

----------------------------------------
АЛГОРИТМ: Fisher-Yates
----------------------------------------

small_case: [4]
  total_permutations_found: 24
  expected_permutations: 24
  chi_squared: 27.4352
  expected_count_per_permutation: 416.67
  coverage_ratio: 1.0
  uniformity_quality: poor

medium_case: [10]
  expected_count_per_position: 1000.0
  avg_chi_squared: 11.595
  max_chi_squared: 28.204
  min_chi_squared: 4.008
  position_uniformity: poor

----------------------------------------
АЛГОРИТМ: Generate-Permutation
----------------------------------------

small_case: [4]
  total_permutations_found: 24
  expected_permutations: 24
  chi_squared: 32.5952
  expected_count_per_permutation: 416.67
  coverage_ratio: 1.0
  uniformity_quality: poor

medium_case: [10]
  expected_count_per_position: 1000.0
  avg_chi_squared: 9.4484
  max_chi_squared: 15.448
  min_chi_squared: 5.144
  position_uniformity: good

ТЕСТ ПЕР